# Active Learning Deep Gaussian Process Regression Monte Carlo Simulation

https://docs.gpytorch.ai/en/latest/examples/01_Exact_GPs/Simple_GP_Regression.html

https://docs.gpytorch.ai/en/stable/examples/06_PyTorch_NN_Integration_DKL/KISSGP_Deep_Kernel_Regression_CUDA.html

In [1]:
import numpy as np
import torch
from scipy.special import ndtri
import random

from core.SNDGPR.train import train_model
from core.MCS import MC_sampling_plan, MC_prediction, estimate_Pf
from core.bay_opt.K_fold_train import kfold_train
from core.bay_opt.bayesian_optimization import bayesian_optimization
from core.bay_opt.optimization_variables import optimization_variables

from core.utils import load_core_modules, load_example_modules, sample_info, print_info, \
    save_bests, plot_losses, min_max_normalization, save_x_added, evaluate_g, \
    pickle_save, pickle_load, results_plot, results_print

In [2]:
# Set the seed for reproducibility
SEED = 42

# For NumPy
np.random.seed(SEED)

# For PyTorch
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # if you are using multi-GPU

# For Python's built-in random module
random.seed(SEED)

# Ensuring reproducibility in cuDNN using PyTorch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Set parameters

In [3]:
N_MC = 1e6  # number of points for Monte Carlo
TRAINING_ITERATIONS = 1000  # epochs for training the DGPR
LEARNING_RATE = 0.01  # learning rate for training the model with Adam
VALIDATION_SPLIT = 6  # K-Fold split (K = VALIDATION_SPLIT)
N = 81  # initial number of points
N_INFILL = 500  # number of infill points
ALPHA = 0.05  # alpha for confidence bounds
SPECTRAL_NORMALIZATION = True
EXAMPLE = 'example1'  # example to run
SAMPLING_PLAN_STRATEGY = 'LHS'  # initial sampling plan generation strategy
LEARNING_FUNCTION = 'U'  # learning function
CONVERGENCE_FUNCTION = 'stop_Pf'  # stopping criteria

Set parameters of Bayesian Optimization

In [4]:
N_INITIAL_EGO = 25
N_INFILL_EGO = 15

DIM_EGO = 3  # number of hyperparameters to optimize
TRAINING_ITERATIONS_EGO = 10000
BOUNDS_BSA = (
    tuple((0, 1) for _ in range(DIM_EGO))  # the number of (0, 1) tuples has to be equal to DIM
    )
BSA_POPSIZE = 20
BSA_EPOCH = 200
# Adjust BSA population size and epochs based on the size of your search space,
# defined by BOUNDS_BAY_OPT.
# Make BSA_POPSIZE * BSA_EPOCH much larger than your search space.
BOUNDS_BAY_OPT = [[1, 5], [1, 5], [0, 4]]  # L, r, act_fun
# For more activation functions, check optimization_variables.py
# and adjust the nonlinearity of the weight initialization in SNDGPR.py

Load random variables and limit state function dynamically

In [5]:
RVs, limit_state_function = load_example_modules(EXAMPLE)

Load core functions dynamically

In [6]:
initial_sampling_plan, learning_function, evaluate_lf, convergence_function = load_core_modules(
    SAMPLING_PLAN_STRATEGY, LEARNING_FUNCTION, CONVERGENCE_FUNCTION
)

In [7]:
Params = dict()
Params['RVs'] = RVs
Params['N_MC'] = N_MC
Params['training_iterations'] = TRAINING_ITERATIONS
Params['N'] = N
Params['N_added'] = N_INFILL
Params['alpha'] = ALPHA
Params['limit_state_function'] = EXAMPLE
Params['initial_sampling_plan'] = SAMPLING_PLAN_STRATEGY
Params['learning_function'] = LEARNING_FUNCTION
Params['convergence_function'] = CONVERGENCE_FUNCTION
Params['seed'] = SEED
Params['SN'] = SPECTRAL_NORMALIZATION

Params['N_EGO'] = N_INITIAL_EGO
Params['N_INFILL_EGO'] = N_INFILL_EGO
Params['DIM_EGO'] = DIM_EGO
Params['TRAINING_ITERATIONS_EGO'] = TRAINING_ITERATIONS_EGO
Params['BSA_POPSIZE'] = BSA_POPSIZE
Params['BSA_EPOCHS'] = BSA_EPOCH
Params['BOUNDS_BAY_OPT'] = BOUNDS_BAY_OPT

### Sample points

In [8]:
x = initial_sampling_plan(N, RVs, SEED)

In [9]:
x_candidate = MC_sampling_plan(N_MC, RVs)

In [10]:
sample_info(RVs, [SAMPLING_PLAN_STRATEGY, 'Monte Carlo'], [x, x_candidate])

RV: x_1 -- mean: 1.00, std: 0.10, distribution: normal
LHS sample -- mean: 1.00, std: 0.10, max: 1.28, min: 0.77
Monte Carlo sample -- mean: 1.00, std: 0.10, max: 1.46, min: 0.51

RV: x_2 -- mean: 1.00, std: 0.10, distribution: normal
LHS sample -- mean: 1.00, std: 0.10, max: 1.33, min: 0.74
Monte Carlo sample -- mean: 1.00, std: 0.10, max: 1.48, min: 0.51

RV: x_3 -- mean: 1.00, std: 0.10, distribution: normal
LHS sample -- mean: 1.00, std: 0.10, max: 1.24, min: 0.74
Monte Carlo sample -- mean: 1.00, std: 0.10, max: 1.46, min: 0.52

RV: x_4 -- mean: 1.00, std: 0.10, distribution: normal
LHS sample -- mean: 1.00, std: 0.10, max: 1.27, min: 0.72
Monte Carlo sample -- mean: 1.00, std: 0.10, max: 1.51, min: 0.51

RV: x_5 -- mean: 1.00, std: 0.10, distribution: normal
LHS sample -- mean: 1.00, std: 0.10, max: 1.32, min: 0.67
Monte Carlo sample -- mean: 1.00, std: 0.10, max: 1.45, min: 0.51

RV: x_6 -- mean: 1.00, std: 0.10, distribution: normal
LHS sample -- mean: 1.00, std: 0.10, max: 1.2

Evaluate initial sampling plan, $\Gamma$

In [11]:
g = limit_state_function(x)

In [12]:
print(g)

tensor([-0.4605,  5.3109,  5.5517,  3.9244,  7.5034,  5.6346,  5.8629,  6.1103,
         4.9282,  4.2455,  5.3963,  7.0045,  6.4977,  4.1306,  5.2996,  6.6697,
         5.6219,  4.8630,  7.0146,  5.9483,  4.3584,  6.9780,  3.0722,  5.4441,
         1.5697,  3.7908,  3.9184,  6.1917,  6.2909,  6.6146,  3.7923,  4.5557,
         8.6837,  5.3400,  4.7518,  9.6464,  6.2588,  2.8204,  6.0438,  3.6143,
         4.1308,  3.6735,  5.8060,  3.6570,  4.8131,  8.3549,  3.8824,  5.4858,
         3.3773,  5.0653,  6.5012,  4.5397,  7.1606,  2.7451,  5.2270,  5.9716,
         4.8779,  8.1504,  3.2944,  6.9476,  3.8873,  6.8027,  6.6847, 10.2823,
         6.1027,  8.9382,  6.9725,  6.7453,  4.8158,  3.9147,  8.1673,  3.3884,
         7.4162,  7.9842,  6.2344,  1.9904,  6.1730,  6.1915,  3.2288,  5.5694,
         4.6589])


## Active learning

In [13]:
data_dim = x.shape[1]
estimate_Pf_all = []
estimate_Pf_allp = []
estimate_Pf_allm = []
estimate_N_samples_added = []
N_samples_added_total = 0
converged = False
it = 0

In [14]:
while True:
    print(f'\nIteration {it}')
    
    if it == 0:
        f_EGO, x_EGO, model, likelihood, train_losses, val_losses, train_x, val_x, train_g, val_g, x_max, x_min \
            = bayesian_optimization(
                x, g, x_candidate, N, N_MC, ALPHA, N_INITIAL_EGO, N_INFILL_EGO, DIM_EGO, \
                TRAINING_ITERATIONS_EGO, BOUNDS_BSA, BSA_POPSIZE, BSA_EPOCH, \
                SEED, TRAINING_ITERATIONS, BOUNDS_BAY_OPT, SPECTRAL_NORMALIZATION, \
                VALIDATION_SPLIT, LEARNING_RATE, Params
                )
            
    else:
        layer_sizes, act_fun = optimization_variables(BOUNDS_BAY_OPT, x_EGO[torch.argmin(f_EGO), :], x, SPECTRAL_NORMALIZATION)
        
        _, _, model, likelihood, train_losses, val_losses, train_x, val_x, train_g, val_g, x_max, x_min, fold \
            = kfold_train(
            x, g, x_candidate, TRAINING_ITERATIONS, LEARNING_RATE, layer_sizes, act_fun, \
                N, N_MC, ALPHA, SPECTRAL_NORMALIZATION, Params, n_splits=VALIDATION_SPLIT, SEED=SEED
            )
    
    # Save variables and plot loss
    save_bests(model, likelihood, train_losses, val_losses, x, train_x, val_x, train_g, val_g,
               x_EGO, f_EGO, x_max, x_min, it, BOUNDS_BAY_OPT, SPECTRAL_NORMALIZATION, EXAMPLE)
    plot_losses(train_losses, val_losses, it)
    
    # Predict MC responses (only the sample which are not contained in the Kriging yet)
    x_candidate_normalized = min_max_normalization(x_max, x_min, x_candidate)
    preds = MC_prediction(model, likelihood, x_candidate_normalized)
    
    # Evaluate learning function
    g_mean, gs, ind_lf = evaluate_lf(preds, learning_function)
    
    # Select additional sample (the sample which maximizes the learning function value)
    x_added = x_candidate[ind_lf, :]
    x_added = x_added.view(1, data_dim)
    # x_added = x_added * (x_max - x_min) + x_min  # undo normalization
    
    save_x_added(x_added, it, EXAMPLE)  # Salve o array do NumPy em um arquivo .mat
    
    # Estimate Pf
    Pf, Pf_plus, Pf_minus = estimate_Pf(g, g_mean, gs, N, N_MC, ALPHA)
    
    estimate_Pf_all.append(Pf)
    estimate_Pf_allp.append(Pf_plus)
    estimate_Pf_allm.append(Pf_minus)
    estimate_N_samples_added.append(N_samples_added_total)
    
    # Print some info
    print_info(N, N_INFILL, it, Pf, Pf_plus, Pf_minus)
    
    # Check if maximum number of points were added
    if N_samples_added_total >= N_INFILL: break
    it += 1
    
    # Convergence criterion
    if converged and N_samples_added_total != 0:
        if convergence_function(g, g_mean, gs, N, N_MC): break
        converged = False
    else:
        converged = convergence_function(g, g_mean, gs, N, N_MC)
    
    g_added = evaluate_g(x_added, it, limit_state_function, EXAMPLE)
    
    x = torch.cat((x, x_added), 0)
    g = torch.cat((g, g_added), 0)
    x_candidate = torch.cat((x_candidate[:ind_lf], x_candidate[ind_lf+1:]))
    N_samples_added_total = N_samples_added_total + 1


Iteration 0
Hyperparameters: [35, 15, 6, 2], SN: True, act_fun: Tanh
Early stopping at epoch 150
Best Loss: 1.984727919101715 at epoch 48. Training loss: 1.447919249534607 and val. loss: 2.5215365886688232
  Training succeeded after 1 attempt(s).


c:\Users\T-GAMER\Desktop\DOC\Artigo SN\AL-SNDGPR-MCS\core\SNDGPR\train.py:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("best_model_and_likeliho

avg. loss = 1.9847, delta = inf%, Pf: [1.8000000636675395e-05, 0.0, 0.0]
Fold: 1, Avg. Loss: 1.984727919101715

Early stopping at epoch 217
Best Loss: 1.489379107952118 at epoch 115. Training loss: 1.3867563009262085 and val. loss: 1.5920019149780273
  Training succeeded after 1 attempt(s).
avg. loss = 1.4894, delta = inf%, Pf: [0.0009539999882690609, 0.0, 0.0]
New best model found at fold 2:
    delta = inf%, avg. loss = 1.4894
    layer_sizes: [35, 15, 6, 2], act fun: <class 'torch.nn.modules.activation.Tanh'>

Fold: 2, Avg. Loss: 1.489379107952118

Early stopping at epoch 203
Best Loss: 1.615769386291504 at epoch 101. Training loss: 1.443704605102539 and val. loss: 1.7878341674804688
  Training succeeded after 1 attempt(s).
avg. loss = 1.6158, delta = inf%, Pf: [0.00072900002123788, 0.0, 0.0]
Fold: 3, Avg. Loss: 1.615769386291504

Early stopping at epoch 231
Best Loss: 1.3860222101211548 at epoch 129. Training loss: 1.2892876863479614 and val. loss: 1.4827567338943481
  Training suc

KeyboardInterrupt: 

## Store results

In [38]:
# Store results
# Estimate failure probability
estimate_Pf_0 = (torch.sum(g_mean <= 0) + torch.sum(g[N+1:] <= 0))/N_MC

# Estimate the covariance
estimate_CoV = torch.sqrt((1-estimate_Pf_0) / estimate_Pf_0 / N_MC)

# Store the results
Results = {
    'Pf': estimate_Pf_0,
    'Beta': -ndtri(estimate_Pf_0),
    'CoV': estimate_CoV,
    'Model_Evaluations': N_samples_added_total + N,
    'Pf_CI': estimate_Pf_0 * np.array([
        1 + ndtri(ALPHA/2)*estimate_CoV,
        1 + ndtri(1-ALPHA/2)*estimate_CoV
        ]),
    }
Results['Beta_CI'] = torch.flip(-ndtri(Results['Pf_CI']), [0])

History = {
    'Pf': estimate_Pf_all,
    'Pf_Upper': estimate_Pf_allp,
    'Pf_Lower': estimate_Pf_allm,
    'N_Samples': estimate_N_samples_added,
    'N_Init': N,
    'X': x,
    'G': g,
    'MC_Sample': x_candidate,
}

## SAVE

In [40]:
pickle_save(Results, History, Params, EXAMPLE)

## Display results and plot

In [ ]:
results_print(Results, History, Params)

In [ ]:
results_plot(Results, History, Params, EXAMPLE)

## LOAD

In [44]:
# Results, History, Params = pickle_load(EXAMPLE)